# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125911e+02     1.525967e+00
 * time: 0.12418484687805176
     1     1.049991e+01     8.494469e-01
 * time: 1.3621690273284912
     2    -1.159584e+01     9.483355e-01
 * time: 1.4972529411315918
     3    -3.425352e+01     7.672246e-01
 * time: 1.6563358306884766
     4    -4.775584e+01     5.856345e-01
 * time: 1.8102929592132568
     5    -5.710598e+01     1.818976e-01
 * time: 1.9553618431091309
     6    -5.994199e+01     1.047006e-01
 * time: 2.063232898712158
     7    -6.092462e+01     5.137782e-02
 * time: 2.1780569553375244
     8    -6.128413e+01     6.349415e-02
 * time: 2.284824848175049
     9    -6.160034e+01     3.152810e-02
 * time: 2.3999478816986084
    10    -6.181510e+01     3.105613e-02
 * time: 2.509462833404541
    11    -6.198184e+01     2.546497e-02
 * time: 2.625082015991211
    12    -6.205229e+01     1.856098e-02
 * time: 2.733719825744629
    13    -6.211792e+01     1.606201e-02
 * time: 2.84989190101623

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522630
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666462732
